In [19]:
import sys
sys.executable


'/Users/surbhisharma/rag_assesment/venv/bin/python'

In [2]:
!pip3 install transformers datasets peft torch


You should consider upgrading via the '/Users/surbhisharma/rag_assesment/services/embedding_service/venv/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip3 install ipykernel



  Using cached ipykernel-6.30.1-py3-none-any.whl (117 kB)
  Using cached jupyter_client-8.6.3-py3-none-any.whl (106 kB)
  Using cached comm-0.2.3-py3-none-any.whl (7.3 kB)
  Using cached appnope-0.1.4-py2.py3-none-any.whl (4.3 kB)
  Using cached tornado-6.5.2-cp39-abi3-macosx_10_9_x86_64.whl (440 kB)
  Using cached nest_asyncio-1.6.0-py3-none-any.whl (5.2 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl (85 kB)
  Using cached pyzmq-27.0.2-cp39-cp39-macosx_10_15_universal2.whl (1.3 MB)
  Using cached debugpy-1.8.16-cp39-cp39-macosx_14_0_x86_64.whl (2.1 MB)
  Using cached ipython-8.18.1-py3-none-any.whl (808 kB)
  Using cached matplotlib_inline-0.1.7-py3-none-any.whl (9.9 kB)
  Using cached jupyter_core-5.8.1-py3-none-any.whl (28 kB)
  Using cached prompt_toolkit-3.0.52-py3-none-any.whl (391 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl (1.6 MB)
  Using cached stack_data-0.6.3-py3-none-any.whl (24 kB)
  Using cached pexpect-4.9.0-py2.py3-none-any.whl (63 kB)
  Using cached deco

In [10]:
!pip3 install datasets

You should consider upgrading via the '/Users/surbhisharma/rag_assesment/services/embedding_service/venv/bin/python3 -m pip install --upgrade pip' command.


In [6]:
python3 -m ipykernel install --user --name=venv --display-name "Python (venv)"


SyntaxError: invalid syntax (2869923740.py, line 1)

In [8]:
python3


NameError: name 'python3' is not defined

In [24]:
from datasets import Dataset


ModuleNotFoundError: No module named 'datasets'

In [9]:
from datasets import Dataset

data = {
    "question": ["What is the capital of France?", "Who wrote '1984'?", "What is 2 + 2?"],
    "answer": ["Paris", "George Orwell", "4"]
}

dataset = Dataset.from_dict(data)


ModuleNotFoundError: No module named 'datasets'